@author: p@kalaytan.com

This is completed notepad.
Steps:
1. download tick data
2. convert from ticks to renko bars
3. convert to Pandas DataFrames
4. plot using plotly

# Download Ticks
I'm using [findatapy](https://github.com/cuemacro/findatapy) for downloading


In [1]:
import sys
try:
    sys.path.append('C:/Python/findatapy')
except:
    pass
from findatapy.market import Market, MarketDataRequest, MarketDataGenerator

In [2]:
# download and return requested data 
# returns: pandas series with ticks
def download_data(start_date, finish_date, ticker):
    md_request = MarketDataRequest(start_date=start_date, finish_date=finish_date,
                                   category='fx', fields=['bid', 'ask'], freq='tick',
                                   data_source='dukascopy', tickers=[ticker])
    market = Market(market_data_generator=MarketDataGenerator())
    return market.fetch_market(md_request)

In [3]:
ticks = download_data('1 Jan 2016', '31 Jan 2016', 'EURCHF')

2016-12-19 19:19:23,161 - findatapy.market.datavendorweb - INFO - Request Dukascopy data
2016-12-19 19:19:23,162 - findatapy.market.datavendorweb - INFO - About to download from Dukascopy... for EURCHF
2016-12-19 19:19:23,163 - findatapy.market.datavendorweb - INFO - Downloading... 2016-01-01 00:00:00
2016-12-19 19:19:29,037 - findatapy.market.datavendorweb - INFO - Downloading... 2016-01-02 00:00:00
2016-12-19 19:19:34,548 - findatapy.market.datavendorweb - INFO - Downloading... 2016-01-03 00:00:00
2016-12-19 19:19:40,634 - findatapy.market.datavendorweb - INFO - Downloading... 2016-01-04 00:00:00
2016-12-19 19:19:47,133 - findatapy.market.datavendorweb - INFO - Downloading... 2016-01-05 00:00:00
2016-12-19 19:19:53,218 - findatapy.market.datavendorweb - INFO - Downloading... 2016-01-06 00:00:00
2016-12-19 19:19:56,984 - findatapy.market.datavendorweb - INFO - Downloading... 2016-01-07 00:00:00
2016-12-19 19:20:00,774 - findatapy.market.datavendorweb - INFO - Downloading... 2016-01-08

In [4]:
#Let's see what we have
ticks.head(3)

Date
2016-01-01 00:00:01.265    1.109120
2016-01-01 00:00:01.572    1.109100
2016-01-01 00:00:01.881    1.109085
dtype: float32

# Convert Ticks to Renko bars
To prevent some popular misunderstandings, I've created <strong><u>renko bars with ticks and reverse candles</u></strong>

In [5]:
# market and renko var settings
point = 0.00001 
digits = 5
renko_size = 100 # the size of the renko we would like to have
renko_sizeFloat = renko_size * point

In [6]:
# getting low and high borders for 
# in: float - price 
# out: float, float  - hi and low for renko bar for give price
def get_renko_borders(tick):
    low = tick - tick % renko_sizeFloat
    hi = round(low + renko_sizeFloat, digits)
    #print ("tick:{}. hi:{:.5f}. low:{:.5f}".format(tick, hi, low))
    return hi, low

In [7]:
# get first renko bar border for first tick
border_high, border_low = get_renko_borders(ticks[0])
print('first tick:{:.5f}, border_high:{:.5f}, border_low:{:.5f}'.format(ticks[0], border_high, border_low))

first tick:1.10912, border_high:1.11000, border_low:1.10900


In [8]:
# create list where we will hold computed bars
# list will hold: [date, open, high, low, close]
# don't really need date but saving just in case
renko = []

# set all(ohlc) values to first tick
high = low = open = close = ticks[0]

# create first bar with opening tick
renko.append([ticks.index[0], ticks[0], ticks[0], ticks[0], ticks[0]])

In [9]:
# loop through each tick
for i, tick in enumerate(ticks):
    # do we have a bar break? i.e new bar?
    if(tick >= border_high or tick < border_low):
        # update closed bar with latest values
        renko[-1][2] = high
        renko[-1][3] = low
        renko[-1][4] = ticks[i-1]
        
        #prep vars for new renko bar
        open = high = low = close = tick
        
        # add new bar to renko list. [date, open, high, low, close] 
        renko.append([ticks.index[i], open, high, low, close])
       
        # prep borders for next bar
        # if we just closed green candle
        if(tick >= border_high):
            # get new candle borders for new tick
            border_high, border_low = get_renko_borders(open)
            # adjust bottom border in case price reverse
            border_low -= renko_sizeFloat
        # same but opposite
        elif(tick < border_low):
            border_high, border_low = get_renko_borders(open)
            border_high += renko_sizeFloat
        
    #update high and low variables for new tick 
    if(tick > high):
        high = tick
    if(tick < low):
        low = tick

# set latest values to last bar
renko[-1][2] = high
renko[-1][3] = low
renko[-1][4] = ticks[-1]
print(len(renko))

389


In [10]:
# let's see our list
renko[:3]

[[Timestamp('2016-01-01 00:00:01.265000'),
  1.10912,
  1.10922,
  1.109015,
  1.109015],
 [Timestamp('2016-01-01 00:00:45.024000'),
  1.109,
  1.10934,
  1.1080201,
  1.1080201],
 [Timestamp('2016-01-01 00:46:34.457000'),
  1.107995,
  1.1089749,
  1.1075799,
  1.1089699]]

# Convert to Pandas DataFrames

In [11]:
import pandas

In [12]:
# prep columns tuple
columns = ('date', 'open', 'high', 'low', 'close')
df = pandas.DataFrame(renko, columns=columns)

# make our date column index
df.set_index(['date'], inplace = True, drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 389 entries, 2016-01-01 00:00:01.265000 to 2016-01-29 20:53:12.439000
Data columns (total 4 columns):
open     389 non-null float64
high     389 non-null float64
low      389 non-null float64
close    389 non-null float64
dtypes: float64(4)
memory usage: 15.2 KB


In [13]:
df.head(5)

,open,high,low,close
date,,,,
2016-01-01 00:00:01.265,1.109120,1.109220,1.109015,1.109015
2016-01-01 00:00:45.024,1.109000,1.109340,1.108020,1.108020
2016-01-01 00:46:34.457,1.107995,1.108975,1.107580,1.108970
2016-01-01 07:07:08.767,1.109005,1.109070,1.108000,1.108000
2016-01-01 07:38:31.855,1.107965,1.108980,1.107675,1.108980


In [14]:
# we can see our fist tick == to open of first bar
print("First tick = {:.5f} == {:.5f} = first bar open price".format(ticks[0], df.open[0]))
# last tick
print("Last tick = {:.5f} == {:.5f} = last bar close price".format(ticks[-1], df.close[-1]))


First tick = 1.10912 == 1.10912 = first bar open price
Last tick = 1.08639 == 1.08639 = last bar close price


# Plot Renko Bars using Plotly

In [15]:
import plotly.plotly as py
py.sign_in('username', 'key') # get yours at plot.ly
from plotly.tools import FigureFactory as FF

In [16]:
# if we want to use date, add dates=df.index
fig = FF.create_candlestick(df.open, df.high, df.low, df.close)
fig['layout'].update({'title':'{} points EURCHF RENKO BARS'.format(renko_size)})
py.iplot(fig, filename='Renko - EURCHF - 100 points', validate= False)

In [ ]:
# url to the chart: https://plot.ly/~Kalaytan/2.embed

In [17]:
# we could plot tics to triple check. We have to many ticks so I'll plot last 10k ticks.
import plotly.graph_objs as go
trace = go.Scatter(x=ticks.index[-10000:], y=ticks.values[-10000:])
py.iplot([trace], filename='EURCHF - Ticks')